# Enhanced Chatbot with DialoGPT - Complete Implementation
This notebook demonstrates a fully functional AI chatbot using DialoGPT. It includes:
- Dependency installation
- Model loading (DialoGPT)
- A console-based chat loop
- A Streamlit interface for web-based interaction

You can run the console-based loop directly in the notebook, or save the code as `app.py` to run the Streamlit app.

# NEW

In [2]:
!pip install transformers torch datasets faiss-cpu streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import faiss
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the dataset
df = pd.read_csv("UpdatedResumeDataSet.csv")  # Ensure the dataset is uploaded to Colab

# Combine 'category' and 'resume' text for embedding
df["text"] = df["Category"] + " " + df["Resume"]

# Load BERT tokenizer & model for embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to compute embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Use [CLS] token representation

# Compute embeddings for all resumes
embeddings = np.vstack([get_embedding(text) for text in df["text"]])

# Build FAISS index for similarity search
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 Distance
index.add(embeddings)

print("FAISS index built successfully!")


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

FAISS index built successfully!


In [ ]:
import pandas as pd

def chatbot_response(user_query):
    # Check if the query is asking for the count of positions in a category
    if 'how many' in user_query.lower() and 'positions' in user_query.lower():
        # Extract category from the user's query
        category = None
        words = user_query.lower().split()

        # List of categories available in the dataset
        df = pd.read_csv("UpdatedResumeDataSet.csv")

        # Extract all unique categories from the dataset
        categories = df['Category'].unique()

        # Iterate through categories to find the one in the user query
        for cat in categories:
            if cat.lower() in user_query.lower():
                category = cat
                break

        # If a category is found, count the positions
        if category:
            category_count = len(df[df['Category'].str.contains(category, case=False, na=False)])
            return f"There are {category_count} positions in the {category} category."
        else:
            return "Sorry, I couldn't find any category in your query."

    else:
        # If query is not asking for position count, return the resume as usual
        query_embedding = get_embedding(user_query)

        # Perform similarity search to find the closest resume
        _, idx = index.search(query_embedding, 1)  # Retrieve top-1 similar resume
        best_match = df.iloc[idx[0][0]]

        return f"Category: {best_match['Category']}\n\nResume: {best_match['Resume']}"

# Test chatbot function
print(chatbot_response("How many Machine Learning positions are there?"))
print(chatbot_response("How many Data Science positions are there?"))
print(chatbot_response("How many Web Designing positions are there?"))


Sorry, I couldn't find any category in your query.
There are 40 positions in the Data Science category.
There are 45 positions in the Web Designing category.


In [ ]:
%%writefile app.py
import streamlit as st
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("UpdatedResumeDataSet.csv")
df["text"] = df["Category"] + " " + df["Resume"]


# Load model & tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Compute embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

embeddings = np.vstack([get_embedding(text) for text in df["text"]])

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Define chatbot function
def chatbot_response(user_query):
    query_embedding = get_embedding(user_query)
    _, idx = index.search(query_embedding, 1)
    best_match = df.iloc[idx[0][0]]
    return f"Category: {best_match['Category']}\n\nResume: {best_match['Resume']}"

# Streamlit UI
st.title("Resume Chatbot")
user_input = st.text_input("Ask about resumes:", key="user_input")
if user_input:
    response = chatbot_response(user_input)
    st.write(response)




Writing app.py


In [ ]:

!pip install pyngrok
!ngrok config add-authtoken 2tJhEQs7O1eds76HY7PeokUkDan_2VLAkTumosF3Yx7mDBe7A

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#!streamlit run app.py --server.port 8501 &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.133.50:8501



  Stopping...


In [ ]:
#!npx localtunnel --port 8501


⠙⠹⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://loud-colts-hang.loca.lt
^C


In [1]:
!streamlit run app.py --server.headless true --server.port 8501 &


/bin/bash: line 1: streamlit: command not found


In [ ]:

from pyngrok import ngrok
url = ngrok.connect(8501, "http")
print(f"Public URL: {url}")


Public URL: NgrokTunnel: "https://7336-35-221-16-56.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.txt',
 'fine_tuned_model/added_tokens.json',
 'fine_tuned_model/tokenizer.json')

In [ ]:
import os
print(os.listdir("fine_tuned_model"))


['config.json', 'tokenizer_config.json', 'model.safetensors', 'tokenizer.json', 'special_tokens_map.json', 'vocab.txt']


#Back-end

In [ ]:
#!pip install flask flask-ngrok torch transformers
!pip install flask flask-ngrok transformers torch requests


In [ ]:
!which ngrok


/usr/local/bin/ngrok


In [ ]:
!pkill -9 ngrok


In [ ]:
!ngrok authtoken 2tJhEQs7O1eds76HY7PeokUkDan_2VLAkTumosF3Yx7mDBe7A # Replace with your actual token


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!nohup ngrok http 5000 & sleep 5


nohup: appending output to 'nohup.out'


In [ ]:
!apt-get install jq


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
jq is already the newest version (1.6-2.1ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
#!curl -s http://localhost:4040/api/tunnels | jq -r .tunnels[0].public_url

!curl -s http://localhost:4040/api/tunnels | jq -r .tunnels[0].public_url


https://68ad-35-221-16-56.ngrok-free.app


In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os

app = Flask(__name__)

# Check if model path exists
model_uri = "/content/fine_tuned_model"
if not os.path.exists(model_uri):
    raise FileNotFoundError(f"Model directory '{model_uri}' not found. Check the correct path.")

# Load fine-tuned model
try:
    model = AutoModelForSequenceClassification.from_pretrained(model_uri)
    tokenizer = AutoTokenizer.from_pretrained(model_uri)
except Exception as e:
    raise RuntimeError(f"Error loading model: {str(e)}")

def predict_category(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()
        return predicted_label
    except Exception as e:
        return f"Error in prediction: {str(e)}"

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    text = data.get("text", "")

    if not text:
        return jsonify({"error": "No text provided"}), 400

    predicted_category = predict_category(text)

    # Ensure the output is valid
    if isinstance(predicted_category, str) and "Error" in predicted_category:
        return jsonify({"error": predicted_category}), 500

    return jsonify({"category_id": predicted_category})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)


Overwriting flask_script.py


In [ ]:
!pip install safetensors


In [ ]:
%%writefile flask_script.py
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import torch
from safetensors.torch import load_file  # Safetensors loader

app = Flask(__name__)

# Path to your fine-tuned model (SafeTensors format)
model_uri = "/content/fine_tuned_model"  # Path to your model folder

# Load config using AutoConfig
config = AutoConfig.from_pretrained(f"{model_uri}/config.json")

# Manually load the model weights with safetensors
model_weights = load_file(f"{model_uri}/model.safetensors")  # Load model weights

# Initialize the model with the config
model = AutoModelForSequenceClassification.from_config(config)

# Manually assign model weights (SafeTensors format)
model.load_state_dict(model_weights, strict=False)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_uri)

def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    return predicted_label

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    text = data.get("text", "")

    if not text:
        return jsonify({"error": "No text provided"}), 400

    predicted_category = predict_category(text)
    return jsonify({"category_id": predicted_category})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


Overwriting flask_script.py


In [ ]:
!python flask_script.py


2025-02-21 21:20:31.822874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740172831.847883   56740 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740172831.855355   56740 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
 * Serving Flask app 'flask_script'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Feb/2025 21:20:36] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [21/Feb/2025 21:20:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Feb/2025 21:20:53] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [21/Feb/2025 21:24:28] "GET / HTTP/1.1" 404 

In [ ]:
!lsof -i:5000


In [ ]:
import os
print(os.listdir("/content/fine_tuned_model"))


['config.json', 'tokenizer_config.json', 'model.safetensors', 'tokenizer.json', 'special_tokens_map.json', 'vocab.txt']


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Path to save the fine-tuned model
save_path = "/content/fine_tuned_model"

# Save model & tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model saved successfully!")


Model saved successfully!
